---
title: "R Notebook"
output: html_notebook
---

In [ ]:
%%R
#install.packages("tidyverse")
#install.packages("conflicted")
#install.packages("ggraph")
#install.packages("foreach")
#install.packages("doParallel")
#install.packages("bnlearn")
#install.packages("actuar")
#install.packages("BiocManager")
#BiocManager::install("Rgraphviz")
library(tidyverse)
library(igraph)
library(ggraph)
library(foreach)
library(doParallel)
library(conflicted)
library(bnlearn)
library(actuar, include.only = c("rztbinom"))
library(Rgraphviz)
#library(dplyr, include.only = c("select", "mutate")

conflicts_prefer(dplyr::filter)

In [ ]:
%%R
rds_name<-function(title){
  # For naming saved files with a time-stamp
  return(paste0(title,as.numeric(Sys.time()) * 1000, '.rds'))
}
print_grouped_df <- function(grouped_df) {
  # Loop over each group
  groups <- group_split(grouped_df)
  group_vars <- group_vars(grouped_df)
  
  for (i in seq_along(groups)) {
    # Get the current group data
    current_group <- groups[[i]]
    
    # Extract the values of the grouping variables for this group
    group_keys <- current_group[1, group_vars, drop = FALSE]
    
    # Print the group header with key values
    cat("\n==== Group:", paste(names(group_keys), "=", as.character(group_keys), collapse = ", "), "====\n")
    
    # Print the current group
    print(current_group)
  }
}

create_bn_dag <- function(dag) {
    # Turn our dag object into bnlearn compatible object
    net_nodes <- names(dag)

    our.net.dag <- empty.graph(net_nodes)

    net_parents <- dag
   
    for (node in net_nodes) {
        if (all(net_parents[node] != "character(0)")) {
        for (single_parent in net_parents[[node]]) {
            our.net.dag <- set.arc(our.net.dag, from = single_parent, to = node)
                }
            }
        }
    return(our.net.dag)
}

options(paged.print = FALSE)

logfactorial<-function(n){
  # Calculate logarithm of factorial of a number
  
  if(n>=2){
    res<-sum(log(seq(2, n)))
  }else{
    res<-0
  }
  return(res)
}


Prod_j <- function(df, i, parents, r) {
  # Helper Function for f_20: Prod_j
  
  #print(paste("Running Prod_j with i =", i))
  #print(paste("Parents:", paste(parents, collapse = ", ")))
  
  if (!is.null(parents) & length(parents) != 0) {
    #print("Grouping by parents...")
    ret <- df %>%
      group_by(across(all_of(parents))) %>%
      summarise(N_ij = n(), .groups = "keep") %>% 
      mutate(ratio= logfactorial(r[i] - 1)- logfactorial(N_ij + r[i] - 1))
  } else {
    #print("No parents to group by...")
    ret <- df %>%
      summarise(N_ij = n(), .groups = "keep") %>% 
      mutate(ratio= logfactorial(r[i] - 1) - logfactorial(N_ij + r[i] - 1))
  }
  
  #print("Intermediate result of Prod_j:")
  #print(ret)
  
  result <- sum(ret$ratio)
  #print(paste("Result of Prod_j:", result))
  return(result)
}


Prod_jk <- function(df, i, parents) {
  # Helper Function for f_20: Prod_jk
  
  #print(paste("Running Prod_jk with i =", i))
  #print(paste("Parents:", paste(parents, collapse = ", ")))
  
  if (length(c(parents, i)) != 0) {
    #print("Grouping by parents and i...")
    ret <- df %>%
      group_by(across(all_of(c(parents, i)))) %>%
      summarise(count = n(), .groups = "keep") %>%
      mutate(factorial_count = logfactorial(count))
    
    #print("Intermediate result of Prod_jk:")
    #print(ret)
    
    result <- sum(ret$factorial_count)
    #print(paste("Result of Prod_jk:", result))
    return(result)
  } else {
    #print("No grouping needed for Prod_jk.")
    return(1)
  }
}

f_20 <- function(original_df, i, parents, r) {
  # Calculates the K2-score of a node given a dataframe, the index of a node, and the vector r_i of dinstict values for each i-th column
  
  #print(paste("Running f_20 with i =", i))
  #print(paste("Parents:", paste(parents, collapse = ", ")))
  
  prod_g_ij <- Prod_j(original_df, i, parents, r)
  prod_f_ijk <- Prod_jk(original_df, i, parents)
  
  #print(paste("prod_g_ij:", prod_g_ij))
  #print(paste("prod_f_ijk:", prod_f_ijk))
  
  result <- prod_g_ij + prod_f_ijk 
  #print(paste("Result of f_20:", result))
  
  return(result)
}


k2_algorithm <- function(data, nodes=NULL, max_parents = NULL) {
  
  # Return list with best network and associated score, given:
  # - a dataframe
  # - a vector of character nodes (optional; in absence, gets ordering from the dataframe columns ordering)
  # - a maximum number of parents (optional; in absence, assumes the as maximum number the total number of nodes - 1)

  if(is.null(nodes)){
    nodes<-names(data)
  }
  if(is.null(max_parents)){
    max_parents<-length(nodes)-1
  }
  # Initialize the network
  network <- setNames(vector("list", length(nodes)), nodes)
  network[[nodes[1]]]<-character(0)
  
  # Detect the number of available cores
  numCores <- parallel::detectCores()
  # Create a cluster with the desired number of cores
  cl <- makeCluster(numCores - 1)  # Use one less core than available
  # Register the cluster for parallel processing
  registerDoParallel(cl)
  # Ensure cluster is stopped even if an error occurs
  on.exit(stopCluster(cl))
  #Calculate r_i
  r<-map_int(data, n_distinct)
  
  # Iterate over each node. This is parallelized according to available number of cores
  result<-foreach (i = 2:length(nodes), .combine = 'c', .packages = c('dplyr', 'purrr'), .export = c('logfactorial', 'Prod_jk', 'Prod_j', 'f_20')) %dopar% {
     # Calculate r_i:

    parents <- c()
    score_old <- f_20(data, i, parents, r)
    proceed <- TRUE
    while (proceed & length(parents) < max_parents) {

      # Find the best candidate parent
      best_parent <- NULL
      best_score <- score_old

      for (z in base::setdiff(1:(i-1), parents)) {
        parents_trial<-c(parents, z)
        best_score <- f_20(data, i, parents_trial, r)
        if (best_score > score_old) { 
          best_parent <- z
          score_old <- best_score
        }
      }
      # Update parents and score if necessary
      if (!is.null(best_parent)) {
        parents <- c(parents, best_parent)
        score_old <- best_score
      } else {
        proceed <- FALSE
      }
    }
    # Return the parents for the current node
    return(list(list(node = nodes[i], foundparents = if(length(parents) == 0) character(0) else sapply(parents, function(el) nodes[el]), best_score=best_score)))
  }
  # Assign results to the network
  total_best_score<-0
  for (res in result) {
    network[[res$node]] <- res$foundparents
    total_best_score<-total_best_score+res$best_score
  }
  
  return(list(dag=network, score=total_best_score))
}

In [ ]:
%%R
print_network<-function(network, layout = "auto"){ 
  # Create dataframe with all nodes
  nodes <- tibble(node = names(network)) 
   
  # Create dataframe with existing arcs (nodes_non_isolated <- V(graph)[degree(graph) > 0]) 
  edges_existing <- tibble( 
    from = unlist(network), 
    to = rep(names(network), sapply(network, length)) 
  ) 
   
  # Identify isolated nodes 
  nodes_isolated <- nodes %>% 
    filter(!(node %in% c(edges_existing$from, edges_existing$to))) 
   
  # Creates dataframe with self-referenced arcs for isolated nodes (needed for plotting technical reasons)
  edges_isolated <- tibble( 
    from = nodes_isolated$node, 
    to = nodes_isolated$node 
  ) 
     
  # Combines the two dataframes 
  edges <- bind_rows(edges_existing, edges_isolated) 
 
  # DAG creation
  graph <- graph_from_data_frame(edges, directed = TRUE) 
  ggraph(graph, layout = layout) + 
    geom_node_point(shape = 21, size = 20, color = "green") + 
    geom_node_text(aes(label = name), repel = FALSE, fontface = "bold", nudge_x = +0, nudge_y = +0) + 
    geom_edge_link(arrow = arrow(length = unit(0.2, "inches"), type = "closed"), 
                   end_cap = circle(), start_cap = circle())
                   #, curve = 0.2) + 
    #labs(title = "Rete Bayesiana") 
}

In [ ]:
%%R
plot_dag<-function(dag){
  # Plots dag with graphviz library
  graphviz.plot(create_bn_dag(dag))
}

Simple datasets for testing:

In [ ]:
%%R
trial_df <- data.frame(
  x1 = c(1, 1, 0, 1, 0, 0, 1, 0, 1, 0),
  x2 = c(0, 1, 0, 1, 0, 1, 1, 0, 1, 0),
  x3 = c(0, 1, 1, 1, 0, 1, 1, 0, 1, 0)
)

trial_df_zeros <- data.frame(
  x1 = c(0, 1, 0, 1, 0, 0, 0, 0, 1, 0),
  x2 = c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
  x3 = c(0, 1, 0, 1, 0, 1, 1, 0, 1, 0),
  x4 = c(0, 1, 1, 1, 0, 1, 1, 0, 1, 0)
)


Test k2_algorithm on simple dataset:

In [ ]:
%%R
network<-k2_algorithm(trial_df)
print(network[['dag']])
print(network[['score']])
print_network(network[['dag']])

Now we load and explore three dataset, used in the following analysis. 
Respectively: asia, child and sachs.

Asia

In [ ]:
%%R
asia <- read_csv('https://www.ccd.pitt.edu/wiki/images/ASIA10k.csv')
head(asia)
asia <- asia |> select(asia, smoke, tub, lung, either, bronc, xray, dysp)

# This is the ground-truth expected network associated to the dataset.
true_dag.asia<- model2network("[asia][smoke][tub|asia][lung|smoke][bronc|smoke][dysp|bronc:either][either|tub:lung][xray|either]")

Child

In [ ]:
%%R
child <- read_csv('https://www.ccd.pitt.edu/wiki/images/CHILD10k.csv')
child <- child |> select(BirthAsphyxia, Disease, Sick, DuctFlow, CardiacMixing, LungParench, LungFlow, LVH, Age, Grunting, HypDistrib, HypoxiaInO2, CO2, ChestXray, LVHreport, GruntingReport, LowerBodyO2, RUQO2, CO2Report, XrayReport)
head(child)

# This is the ground-truth expected network associated to the dataset
true_dag.child<-model2network("[BirthAsphyxia][Disease|BirthAsphyxia][Sick|Disease][DuctFlow|Disease][CardiacMixing|Disease][LungParench|Disease][LungFlow|Disease][LVH|Disease][Age|Disease:Sick][Grunting|Sick:LungParench][HypDistrib|DuctFlow:CardiacMixing][HypoxiaInO2|CardiacMixing:LungParench][CO2|LungParench][ChestXray|LungParench:LungFlow][LVHreport|LVH][GruntingReport|Grunting][LowerBodyO2|HypDistrib:HypoxiaInO2][RUQO2|HypoxiaInO2][CO2Report|CO2][XrayReport|ChestXray]")

Sachs

In [ ]:
%%R
sachs <- read_csv('https://www.ccd.pitt.edu/wiki/images/SACHS10k.csv')
sachs <- sachs |> select(PKC, Plcg, PKA, PIP3, Raf, Jnk, P38, PIP2, Mek, Erk, Akt)
head(sachs)

# This is the ground-truth expected network associated to the dataset
true_dag.sachs <- model2network("[PKC][PKA|PKC][Raf|PKC:PKA][Mek|PKC:PKA:Raf][Erk|Mek:PKA][Akt|Erk:PKA][P38|PKC:PKA][Jnk|PKC:PKA][Plcg][PIP3|Plcg][PIP2|Plcg:PIP3]")

We defined for global usage some list and vectors associated to important features of each dataset (name, expected DAG, maximum number of parents present, univocal color for clearer plotting):

In [ ]:
%%R
dataset_names<-c('asia', 'child', 'sachs')
datasets<-list(asia=asia, child=child, sachs=sachs)
true_dags<-list(asia=true_dag.asia, child=true_dag.child, sachs=true_dag.sachs)
true_max_parents<-list(asia=2, child=2, sachs=3)
dataset_colors<-c(asia='goldenrod', child='seagreen', sachs='firebrick')

##Let's first constrain the nodes ordering of each network according to the ground truth DAG's expected.

###Let's first test our k2 algorithm plotting the resulted best network, for all datasets.
Constrained max number of parents according to ground truth expected DAG's

In [ ]:
%%R
for(dataset_name in dataset_names){
  network<-k2_algorithm(datasets[[dataset_name]], max_parents = true_max_parents[[dataset_name]] )
  plot_dag(network[['dag']])
  #graphviz.compare(true_dags[[dataset_name]], create_bn_dag((network[['dag']])))
}

Unconstrained max number of parents.

In [ ]:
%%R
for(dataset_name in dataset_names){
  network<-k2_algorithm(datasets[[dataset_name]] )
  plot_dag(network[['dag']])
  #graphviz.compare(true_dags[[dataset_name]], create_bn_dag((network[['dag']])))
}

###Comparison between predicted networks and ground truth (expected) networks, as a function of dataset size.
Error is calculated as the complementary of the F1-score.

In [ ]:
%%R
f1_error<-function(my_dag, true_dag, n_true_edges=NULL){
  # Calculate "error" with respect to assumed "true" DAG as 1 - F1_score
  
  if(is.null(n_true_edges)){
    n_true_edges<-nrow(arcs(true_dag))
  }
  dag_compare <- bnlearn::compare(true_dag, my_dag) 
  precision<-as.numeric(dag_compare["tp"])/(as.numeric(dag_compare["tp"])+as.numeric(dag_compare["fp"])) 
  recall<-as.numeric(dag_compare["tp"])/n_true_edges  
  f1<-2*precision*recall/(precision+recall) 
  return(1-f1) 
}

In [ ]:
%%R
error_vs_data_size<-function(network_learner, data_df, true_dag, logscale=FALSE){
  # Plot (and return x,y coordinates of the plot) error as a function of increasing dataset size
  # Arguments are:
  # - network learner: k2_algorithm or alternative with same interface
  # - data_df: dataset as a dataframe
  # - true_dag: expected, "ground truth", DAG associated to dataset
  # - logscale (optional): set logscale for x-axis
  
  spanned=c(10,20,30,50,100,200,300, 500, 1000, 5000, 10000)
  errors=vector('list', length(spanned))
  for(i in 1:length(spanned)){
    my_dag<-create_bn_dag(network_learner(head(data_df, spanned[i]))[['dag']])
    errors[[i]]<-f1_error(my_dag, true_dag)
  }
  if(logscale){
    plot(spanned[1:length(errors)],errors,xlab='Dataset size (logscale)',ylab='Error',type='l',col='blue', log='x')
  }else{
    plot(spanned[1:length(errors)],errors,xlab='Dataset size',ylab='Error',type='l',col='blue') 
  }
  return(list(x=spanned[1:length(errors)], y=errors))
}

First case: initialization with expected nodes ordering and expected max number of parents.

In [ ]:
%%R
constrained_nodes_parents_error_vs_datasize<-setNames(vector("list", 3), c("asia", "child", "sachs"))
CNPEVD<-constrained_nodes_parents_error_vs_datasize
for( dataset_name in c('asia','child','sachs')){
  CNPEVD[[dataset_name]]<-error_vs_data_size(function(data)(k2_algorithm(data, max_parents = 2)), datasets[[dataset_name]], true_dags[[dataset_name]], logscale=TRUE)
}
saveRDS(CNPEVD, file=rds_name('CNPEVD'))

Second case: initialization with expected nodes ordering but 'unconstrained' number of parents.

In [ ]:
%%R
#constrained_nodes_error_vs_datasize<-error_vs_data_size(k2_algorithm, child, true_dag.child)
constrained_nodes_error_vs_datasize<-setNames(vector("list", 3), c("asia", "child", "sachs"))
CNEVD<-constrained_nodes_error_vs_datasize
for( dataset_name in c('asia','child','sachs')){
  CNEVD[[dataset_name]]<-error_vs_data_size(k2_algorithm, datasets[[dataset_name]], true_dags[[dataset_name]], logscale=TRUE)
}
saveRDS(CNEVD, file=rds_name('CNEVD'))

Plot all datasets in one:

Constrained parents:

In [ ]:
%%R
spanned <- c(10, 20, 30, 50, 100, 200, 300, 500, 1000, 5000, 10000) 
endrange <- 9 

# Set graphical parameters
par(mgp = c(1.5, 0.5, 0))  # Adjust mgp to reduce space (default is c(3, 1, 0)) 
# Plot the 'asia' dataset with filled dots 
plot(CNPEVD[['asia']][['x']][1:endrange],  
     CNPEVD[['asia']][['y']][1:endrange],  
     type='b',  
     ylim=c(0,1),  
     ylab="Error",  
     xlab="Dataset size",  
     col='goldenrod',  
     pch=16)  # pch=16 for filled circles 
 
# Define datasets and colors 
dataset_name <- c('child', 'sachs') 
cols <- c("seagreen", "firebrick") 
 
# Loop through and add lines for 'child' and 'sachs' with filled dots 
for(j in 1:length(dataset_name)) { 
  lines(CNPEVD[[dataset_name[j]]][['x']][1:endrange],  
        CNPEVD[[dataset_name[j]]][['y']][1:endrange],  
        col=cols[j],  
        type='b',  
        pch=16)  # pch=16 for filled circles 
} 

# Add legend with matching colors 
legend(x='topright', legend=c('asia', 'child', 'sachs'), col=c('goldenrod', 'seagreen', 'firebrick'), lty=1, pch=16)

Unconstrained parents:

In [ ]:
%%R
spanned <- c(10, 20, 30, 50, 100, 200, 300, 500, 1000, 5000, 10000) 
endrange <- 9 

# Set graphical parameters
par(mgp = c(1.5, 0.5, 0))  # Adjust mgp to reduce space (default is c(3, 1, 0)) 
# Plot the 'asia' dataset with filled dots 
plot(CNEVD[['asia']][['x']][1:endrange],  
     CNEVD[['asia']][['y']][1:endrange],  
     type='b',  
     ylim=c(0,1), 
     ylab="Error",  
     xlab="Dataset size",  
     col='goldenrod',  
     pch=16)  # pch=16 for filled circles 
 
# Define datasets and colors 
dataset_name <- c('child', 'sachs') 
cols <- c("seagreen", "firebrick") 
 
# Loop through and add lines for 'child' and 'sachs' with filled dots 
for(j in 1:length(dataset_name)) { 
  lines(CNEVD[[dataset_name[j]]][['x']][1:endrange],  
        CNEVD[[dataset_name[j]]][['y']][1:endrange],  
        col=cols[j],  
        type='b',  
        pch=16)  # pch=16 for filled circles 
} 

# Add legend with matching colors 
legend(x='topright', legend=c('asia', 'child', 'sachs'), col=c('goldenrod', 'seagreen', 'firebrick'), lty=1, pch=16)

## Now we remove the nodes ordering constraint, searching in the permutation space for the highest score dag.


Function that finds best nodes ordering by uniformly sampling the corresponding permutation space.

In [ ]:
%%R
bn_uniform<-function(score_function, data_df, n_iter){
  # Arguments:
  # - score_function: a function like k2_algorithm, that given an ordering and data, gives as output the list(network,score) of the corresponding best bayesian network
  # - data_df: dataset as a dataframe
  # - n_iter: number of random samples from permutation space
  nodes<-names(data_df)
  chain<-vector('list', n_iter)
  for(i in 1:n_iter){
    best_network<-score_function(data_df)
    chain[[i]]<-list(nodes=nodes, dag=best_network[['dag']], score=best_network[['score']])
    nodes<-sample(nodes)
    data_df <- data_df |> select(all_of(nodes))
  }
  max_index<-which.max(lapply(chain, function(el)(el[['score']])))
  return(list(chain=chain, best=chain[[max_index]]))
}

In [ ]:
%%R
n_iter<-5
start.time<-proc.time()
bn_uniform_result<-bn_uniform(k2_algorithm, sachs, n_iter)
end.time<-proc.time()
cat('bn_ordering_sampling duration' ,n_iter, 'iterations: ', end.time-start.time)

In [ ]:
%%R
saveRDS(bn_uniform_result, file=rds_name('unif_ordering'))
object.size(bn_uniform_result[['chain']])
network_dag<-bn_uniform_result[['best']][['dag']]
chain_scores<-sapply(bn_uniform_result[['chain']], function(el)(el[['score']]))
chain_nodes_orderings<-lapply(bn_uniform_result[['chain']], function(el)(el[['nodes']]))
print(bn_uniform_result[['best']][['score']])
print_network(network_dag)

Function (preceded bu some additional helper functions) that finds best nodes ordering with mcmc-metropolis-hastings-like procedure.

In [ ]:
%%R
uniform_transposition_generator<-function(nodes){
  # Generates new nodes vector that differs, from the given one, by one random transposition of two nodes
  
  # Choose two distinct indices randomly
  indices <- sample(seq_along(nodes), 2)
  
  # Swap the elements at these indices
  transposed_nodes <- nodes
  temp <- transposed_nodes[indices[1]]
  transposed_nodes[indices[1]] <- transposed_nodes[indices[2]]
  transposed_nodes[indices[2]] <- temp
  return(transposed_nodes)
}
ztbinom_permutation_generator<-function(nodes,  mean=1,variance=4/5){ 
  # Generates new nodes vector that differs, from the given one, by k random transpositions of two nodes, k drawn from zero-truncated binomial distribution (its parameters given as optional arguments)
  
  p<-(1-(variance/mean)) 
  n<-mean/p 
  n_transpositions<-rztbinom(1,size=n,prob=p) 
  #x<-seq(1,n,by=1) 
  #plot(x,dztbinom(x,size=n,prob=p)) 
  #perform the transposition 
  temp_nodes<-nodes     
  for(j in 1:n_transpositions){ 
    temp_nodes<-uniform_transposition_generator(temp_nodes)   
  } 
  transposed_nodes<-temp_nodes 
  return(transposed_nodes)   
}

bn_mcmc<-function(proposal_generator, score_function, data_df, n_iter){ 
  # Function for MCMC-Metripolis-Hastings-analogue exploration of the nodes ordering space. Return a list with the full MCMC-chain and the best element of the chain. Each element is a list of: nodes vector, dag of best network according to K2, K2-score of such best DAG
  # Arguments:
  # - proposal_generator: a function like uniform_transposition_generator, that given an ordering, randomly samples a new ordering according to some distribution on the permutation space
  # - score_function: a function like k2_algorithm, that given an ordering and data, gives as output the list(network,score) of the corresponding best bayesian network
  # - data_df: dataset as a dataframe
  # - n_iter: number of random samples from permutation space
  nodes<-names(data_df)
  chain<-vector('list', n_iter)
  best_network<-score_function(data_df)
  best_score<-best_network[['score']]
  chain[[1]]<-list(nodes=nodes, dag=best_network[['dag']], score=best_score)
  old_best_score<-best_score
  
  for(i in 2:n_iter){
    
    proposal_nodes<-proposal_generator(nodes)
    proposal_data_df <- data_df |> select(all_of(proposal_nodes))
    proposal_best_network<-score_function(proposal_data_df)
    proposal_best_score<-proposal_best_network[['score']]
    
    if(runif(1) < min(1, exp(best_score-old_best_score))){ #we take exponential because we are dealing with log of probabilities
      nodes<-proposal_nodes
      best_network<-proposal_best_network
    }
    chain[[i]]<-list(nodes=nodes, dag=best_network[['dag']], score=best_network[['score']])
  }
  max_index<-which.max(lapply(chain, function(el)(el[['score']])))
  return(list(chain=chain, best=chain[[max_index]]))
}

In [ ]:
%%R
n_iter<-5
start.time<-proc.time()
bn_mcmc_result<-bn_mcmc(ztbinom_permutation_generator, k2_algorithm, sachs, n_iter)
end.time<-proc.time()
cat('bn_mcmc duration',n_iter, 'iterations: ', end.time-start.time)

In [ ]:
%%R
saveRDS(bn_mcmc_result, file=rds_name('mcmc_ztbin'))
object.size(bn_mcmc_result[['chain']])
network_dag<-bn_mcmc_result[['best']][['dag']]
chain_scores<-sapply(bn_mcmc_result[['chain']], function(el)(el[['score']]))
chain_nodes_orderings<-lapply(bn_mcmc_result[['chain']], function(el)(el[['nodes']]))
print(bn_mcmc_result[['best']][['score']])
print_network(network_dag)

### Function to study how the maximum score obtained in the ordering search increases with the length of the search.

In [ ]:
%%R
print_scores_vs_ordering<-function(chain_scores, chain_nodes_orderings){
  # Sample data
  y <- chain_scores
  x <- chain_nodes_orderings
  #print(x)
  
  # Convert character vectors to single strings
  labels <- sapply(x, function(vec) paste(vec, collapse = " "))
  
  # Create a basic plot
  plot(y, xaxt = "n", xlab = "Nodes permutations", ylab = "Best score", pch = 19, xlim = c(0,length(x)), ylim = c(min(chain_scores), max(chain_scores)))
  
  # Add custom x-axis labels
  #axis(1, at = seq_along(y), labels = labels, las = 2)  # las = 2 rotates labels vertically
}
print_scores_vs_ordering <- function(chain_scores, chain_nodes_orderings, logscale=FALSE) {
  # Sample data
  y <- chain_scores
  x <- chain_nodes_orderings
  
  # Convert character vectors to single strings
  labels <- sapply(x, function(vec) paste(vec, collapse = " "))
  
  # Set graphical parameters
  par(mgp = c(2, 0.5, 0))  # Adjust mgp to reduce space (default is c(3, 1, 0))
  
  
  plot(1:length(y),y, xaxt = "s", xlab = "", ylab = "Best score (log)", 
       pch = 19, xlim = c(0, length(x)), ylim = c(min(chain_scores), max(chain_scores)))
  
  
  # Manually add x-axis label with fine-tuned position using mtext()
  mtext("Nodes permutations", side = 1, line = 1.5)
  # Add custom x-axis labels
  #axis(1, at = seq_along(y), labels = labels, las = 2)  # las = 2 rotates labels vertically
}


In [ ]:
%%R
max_score_vs_iter<-function(chain_scores, logscale=FALSE){ 
  if(logscale){
    plot(1:length(chain_scores),cummax(chain_scores),xlab='n iterations (logscale)',ylab='Running max score (log)',col='blue',type='l', log='x') 
  }else{
    plot(1:length(chain_scores),cummax(chain_scores),xlab='n iterations',ylab='Running max score (log)',col='blue',type='l') 
  }
}

In [ ]:
%%R
print_scores_vs_ordering(chain_scores, chain_nodes_orderings)

In [ ]:
%%R
max_score_vs_iter(chain_scores)

In [ ]:
%%R
bic_score<-function(dag, data){
  return(score(dag , data = data , type = "bic"))
}

### Function to compare the ground truth expected DAG's with the predicted DAG's as function of the length of the ordering search.

In [ ]:
%%R
errors_vs_iter<-function(chain_scores,true_dag,chain_dags){  
  # Function to compare the ground truth expected DAG's with the predicted DAG's as function of the length of the ordering search. Return vector of errors evaluated by f1_error function
  
  n_true_edges<-nrow(arcs(true_dag)) 
  # helper function to find the index of the cummax 
   
  find_index<-function(max_value,chain_scores){ 
      index<-which(chain_scores==max_value) 
      return(index) 
  } 
  
  #compare_dags helper function between the i-th network and true network 
  compare_dags <- function(index,true_dag, our_nets, n_true_edges=NULL) { 
      if(is.null(n_true_edges)){
        n_true_edges<-nrow(arcs(true_dag)) 
      }
      our_net<-our_nets[[index]] 
      our_dag<-create_bn_dag(our_net) 
      return(f1_error(our_dag, true_dag, n_true_edges=n_true_edges))
      #dag_shd <- shd(true_dag, our_dag) 
   
  } 
   
  indexes<-sapply(cummax(chain_scores),find_index,chain_scores) 
   
  #perform the comparison once the index is retrieved 
  errors<-sapply(indexes,function(index, true_dag, our_nets)(compare_dags(index, true_dag, our_nets, n_true_edges = n_true_edges)),true_dag,our_nets=chain_dags) 
  plot(1:length(errors),errors,xlab='Iteration',ylab='Error',type='l',col='blue') 
  return(errors) 
 
} 

### Now we proceed running the searches and storing the results for the different settings presented, and then we show some plots obtained from them.

Unconstrained parents: mcmc and uniform sampling of nodes ordering.

In [ ]:
%%R
unconstrained_bn_mcmc_results<-setNames(vector("list", 3), c("asia", "child", "sachs"))
UBMR<-unconstrained_bn_mcmc_results
for( dataset_name in c('asia','child','sachs')){
  n_iter<-1000
  start.time<-proc.time()
  UBMR[[dataset_name]]<-bn_mcmc(ztbinom_permutation_generator, k2_algorithm, datasets[[dataset_name]], n_iter)
  end.time<-proc.time()
  cat('bn_mcmc', dataset_name ,'duration',n_iter, 'iterations: ', end.time-start.time)
}
saveRDS(UBMR, file=rds_name('UBMR'))

unconstrained_bn_uniform_results<-setNames(vector("list", 3), c("asia", "child", "sachs"))
UBUR<-unconstrained_bn_uniform_results
for( dataset_name in c('asia','child','sachs')){
  n_iter<-1000
  start.time<-proc.time()
  UBUR[[dataset_name]]<-bn_uniform(k2_algorithm, datasets[[dataset_name]], n_iter)
  end.time<-proc.time()
  cat('bn_uniform', dataset_name ,'duration',n_iter, 'iterations: ', end.time-start.time)
}
saveRDS(UBUR, file=rds_name('UBUR'))

Constrained max parents: mcmc and uniform sampling of nodes ordering.

In [ ]:
%%R
constrained_bn_mcmc_results<-setNames(vector("list", 3), c("asia", "child", "sachs"))
CBMR<-constrained_bn_mcmc_results
for( dataset_name in c('asia','child','sachs')){
  n_iter<-1000
  start.time<-proc.time()
  CBMR[[dataset_name]]<-bn_mcmc(ztbinom_permutation_generator, function(data)(k2_algorithm(data, max_parents = true_max_parents[[dataset_name]])), datasets[[dataset_name]], n_iter)
  end.time<-proc.time()
  cat('bn_mcmc', dataset_name ,'duration',n_iter, 'iterations: ', end.time-start.time)
}
saveRDS(CBMR, file=rds_name('CBMR'))

constrained_bn_uniform_results<-setNames(vector("list", 3), c("asia", "child", "sachs"))
CBUR<-constrained_bn_uniform_results
for( dataset_name in c('asia','child','sachs')){
  n_iter<-1000
  start.time<-proc.time()
  CBUR[[dataset_name]]<-bn_uniform(function(data)(k2_algorithm(data, max_parents = true_max_parents[[dataset_name]])), datasets[[dataset_name]], n_iter)
  end.time<-proc.time()
  cat('bn_uniform', dataset_name ,'duration',n_iter, 'iterations: ', end.time-start.time)
}
saveRDS(CBUR, file=rds_name('CBUR'))

## Let's start from a constrained max number of parents:

In [ ]:
%%R
# Loading them after having saved them. (Proceeding this way allowed us to actually ran the previous code in different computers for speeding up the computations, and then analyse all the results in this notebook)

readCBMR<-readRDS('CBMR1725155339414.54.rds')
readCBUR<-readRDS('CBUR1725164868090.98.rds')

PLOTS ON RESULTS OF THE ORDERING SEARCHES

In [ ]:
%%R
C_errors_vs_iter_pack<-list(CBMR= setNames(vector("list", length(dataset_names)), dataset_names), 
                          CBUR=setNames(vector("list", length(dataset_names)), dataset_names))
C_sea_res_rds_s<-list(CBMR=readCBMR, CBUR=readCBUR)

In [ ]:
%%R
extract_auxiliary_1<-function(errors_vs_iter_pack, sea_res_rds_s){
  # Auxiliary function for handy extraction and manipulation of saved results
  
  options(paged.print = FALSE)
  for(sea_res_name in base::names(sea_res_rds_s)){
    sea_res_rds<-sea_res_rds_s[[sea_res_name]]
    for(dataset_name in dataset_names){
      sea_res<-sea_res_rds[[dataset_name]]
      print(str(sea_res))
      object.size(sea_res[['chain']])
      print(network_dag<-sea_res[['best']][['dag']])
      chain_scores<-sapply(sea_res[['chain']], function(el)(el[['score']]))
      chain_dags<-lapply(sea_res[["chain"]],function(el)(el[["dag"]]))
      chain_nodes_orderings<-lapply(sea_res[['chain']], function(el)(el[['nodes']]))
      cat('best score: ', sea_res[['best']][['score']])
      #print_network(network_dag)
      plot_dag(network_dag)
      graphviz.compare(true_dags[[dataset_name]], create_bn_dag(network_dag))
      print_scores_vs_ordering(chain_scores, chain_nodes_orderings)
      max_score_vs_iter(chain_scores, logscale=TRUE)
      #my_best_bic<-bic_score(create_bn_dag(network_dag), datasets[[dataset_name]])
      #true_best_bic<-bic_score(true_dags[[dataset_name]], datasets[[dataset_name]])
      #cat('bic best predicted: ',my_best_bic , 'bic best expected: ',true_best_bic)
      errors_vs_iter_pack[[sea_res_name]][[dataset_name]]<-errors_vs_iter(chain_scores, true_dags[[dataset_name]], chain_dags)
    }
  }
  return(errors_vs_iter_pack)
}

In [ ]:
%%R
plot_auxiliary_1<-function(sea_res_name, errors_vs_iter_pack){
  # Auxiliary function for plotting results of different datasets on same plot
  
  # sea_res_name is for example 'CBMR'
  # Set graphical parameters
  par(mgp = c(1.5, 0.5, 0))  # Adjust mgp to reduce space (default is c(3, 1, 0)) 
  
  ypack<-errors_vs_iter_pack[[sea_res_name]]
  
  endrange <- length(ypack[[1]])
  # Plot the 'asia' dataset with filled dots 
  plot(1:endrange,  
       ypack[['asia']][1:endrange],  
       type='l',  
       ylim=c(min(unlist(ypack)), max(unlist(ypack))),  
       ylab="Error",  
       xlab="n iterations",  
       col='goldenrod',  
       pch=16)  # pch=16 for filled circles 
   
  # Loop through and add lines for 'child' and 'sachs' with filled dots 
  for(dataset_name in c('child', 'sachs') ) { 
    lines(1:endrange,  
          ypack[[dataset_name]][1:endrange],  
          col=dataset_colors[[dataset_name]],  
          type='l',  
          pch=16)  # pch=16 for filled circles 
  } 
  
  # Add legend with matching colors 
  legend(x='topright', legend=c('asia', 'child', 'sachs'), col=c('goldenrod', 'seagreen', 'firebrick'), lty=1, pch=16)
}

In [ ]:
%%R
C_errors_vs_iter_pack<-extract_auxiliary_1(C_errors_vs_iter_pack, C_sea_res_rds_s)

Constrained number of parents, MCMC search (with ztbinom_generator as proposal generator).

In [ ]:
%%R
plot_auxiliary_1('CBMR', C_errors_vs_iter_pack)

Constrained number of parents, uniform search in the permuatation space.

In [ ]:
%%R
plot_auxiliary_1('CBUR', C_errors_vs_iter_pack)

## And finally an unconstrained number of parents:

In [ ]:
%%R
readUBMR<-readRDS('UBMR1725165402263.55.rds')
readUBUR<-readRDS('UBUR1725182729436.31.rds') 

In [ ]:
%%R
U_errors_vs_iter_pack<-list(UBMR= setNames(vector("list", length(dataset_names)), dataset_names), 
                          UBUR=setNames(vector("list", length(dataset_names)), dataset_names))
U_sea_res_rds_s<-list(UBMR=readUBMR, UBUR=readUBUR)

U_errors_vs_iter_pack<-extract_auxiliary_1(U_errors_vs_iter_pack, U_sea_res_rds_s)

Unconstrained number of parents, MCMC search (with ztbinom_generator as proposal generator).

In [ ]:
%%R
plot_auxiliary_1('UBMR', U_errors_vs_iter_pack)

Unconstrained number of parents, uniform search in the permuatation space.

In [ ]:
%%R
plot_auxiliary_1('UBUR', U_errors_vs_iter_pack)

In [ ]:
%%R
save.image(file = "/home/ema/Uni/AdvancedStats/Project/SavedRSessions/my_session.RData") 